This double pendular model of sprinting gait uses assumed values for torsional driving and damping constants.

In [115]:
%matplotlib inline

In [116]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from IPython.display import HTML #for visualization using HTML video

Tibia length: $l_t$

Femur length: $l_f$

Definition of $\vec{r_1}$ and $\vec{r_2}$, where $r_1$ is the position vector of the knee joint and $r_2$ is the position vector of the ankle:

In [121]:
theta_h = 0
omega_h = 0
theta_k = -np.pi/4
omega_phi = 0
m_1 = 2
m_2 = 2
l_f = 2
l_t = 1.5
g = 9.8
t = 0.02
b_1 = 0
b_2 = 0

phi = np.pi - (theta_k - theta_h)

In [122]:
def r_1(l_f, theta_h): #calculates the position vector of knee given a theta
    x = l_f * np.cos(theta_h)
    y = l_f * np.sin(theta_h) # y defined negative in order to hang leg down
    return np.array([x, y])

def r_2(r_1, l_t, theta_k, phi): #calculates the position vector of ankle given a theta    
    x = r_1[0] + l_t * np.cos(phi)
    y = r_1[1] + l_t * np.sin(phi) # y defined negative in order to hang leg down
    return np.array([x, y])

We will look at this model through torques of the individual limbs $l_t$ and $l_f$. In order to do so, we must write down the torque equations:

Again, calling $\phi = \pi - (\theta_k - \theta_h)$:
$$\sum{\tau_1} = l_f m_f g cos({\theta}_h) - T_2 cos (\theta_k - \pi) + k_f \theta_f + b_f \dot{\theta_f} = m_f {l_f}^2 \ddot{\theta_h}$$
$$\sum{\tau_2} = l_t m_t gcos(\phi) + k_t{\theta_t} + b_t{\dot{\theta_t}} = m_t {l_t}^2 \ddot{\phi}$$

$\tau_1$ is dependent on the tension exerted on the femur by the tibia, which we called $T_2$. $T_2$ is really just the centripetal force of the tibia:

$$T_2 = \frac{m_t {v_t}^2}{l_t} = \frac{m_t ({\dot{r_2}})^2}{l_t}$$

$v_2$ is the derivative of $r_2$. Because we are only concerned with the square of velocity:
$$|\dot{r_2}|^2 = l_f^2 \dot{\theta_h}^2 - 2 l_f l_t \dot{\theta_h} \dot{\phi} cos(\theta_k) + l_t^2 \dot{\phi^2}$$

$$l_f m_f g cos({\theta}_h) - T_2 cos (\theta_k - \pi) + k_f \theta_f + b_f \dot{\theta_f} = m_f {l_f}^2 \ddot{\theta_h} = m_f {l_f}^2 \dot{\omega_h}$$

$$l_t m_t gcos(\phi) + k_t{\theta_t} + b_t{\dot{\theta_t}} = m_t {l_t}^2 \ddot{\phi} = m_t {l_t}^2 \dot{\omega_{\phi}}$$

Note that our equations are defined in $\phi$ and not $\theta_k$, although our initial condition is still in terms of $\theta_k$.

In [123]:
def degrees_to_radians(degree): #convert degrees to radians
    rad = degree * (np.pi / 180)
    return rad

def radians_to_degrees(radians): #convert radians to degrees
    deg = radians * (180 / np.pi)
    return deg

def update_system_undamped(theta_h, phi, omega_h, omega_phi, m_1, m_2, l_f, l_t, g, b_1, b_2, t): #including all relevant parameters in function parameters
    I_1 = m_1 * np.square(l_f) # moment of inertia of femur
    I_2 = m_2 * np.square(l_t) # moment of inertia of tibia
    T = (m_2 / l_t) * (np.square(l_f) * np.square(omega_h) + 2 * l_f * l_t * omega_h * omega_phi + np.square(l_t) * np.square(omega_phi)) # tension of the tibia swinging
    
    new_omega_phi = omega_phi - (b_2 / I_2 * omega_phi + (l_t * m_2 * g) / I_2 * np.cos(phi)) * t
    new_phi = phi + new_omega_phi * t

    new_omega_h = omega_h - (b_1 / I_1 * omega_h + (l_f * m_1 * g) / I_1 * np.cos(theta_h) - (l_f / I_1) * T) * t
    new_theta_h = theta_h + new_omega_h * t

    return new_omega_phi, new_phi, new_omega_h, new_theta_h

In [124]:
r_k0 =  r_1(l_f, theta_h) #intial coordinates of knee
r_a0 = r_2(r_k0, l_t, theta_k, phi) #initial coordinates of ankle

#drawing initial limb positions of femur and tibia
x_knee = [0, r_k0[0]]
y_knee = [0, r_k0[1]]

x_ankle = [r_k0[0], r_a0[0]]
y_ankle = [r_k0[1], r_a0[1]]

fig, ax = plt.subplots()
femur, = ax.plot([], [], 'o-', lw=2)
tibia, = ax.plot([], [], 'o-', lw=2)

def animate_undamped(i):
    global theta_h, omega_h, phi, omega_phi, m_1, m_2, l_f, l_t, g, b_1, b_2, t, x_ankle, y_ankle, x_knee, y_knee

    L = l_f + l_t

    r_k =  r_1(l_f, theta_h) #update coordinates of knee
    r_a = r_2(r_k, l_t, theta_k, phi) #update coordinates of ankle

    #update x and y coordinates of knee and ankle
    x_knee[1] = r_k[0]
    y_knee[1] = r_k[1]

    x_ankle = [r_k[0], r_a[0]]
    y_ankle = [r_k[1], r_a[1]]

    omega_phi, phi, omega_h, theta_h = update_system_undamped(theta_h, phi, omega_h, omega_phi, m_1, m_2, l_f, l_t, g, b_1, b_2, t)

    femur.set_data(x_knee, y_knee)
    tibia.set_data(x_ankle, y_ankle)
    ax.set_xlim(-3*L, 3*L)
    ax.set_ylim(-3*L, 3*L)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_title('Undamped Double Pendulum')
    return (femur, tibia)

plt.close()
anim = animation.FuncAnimation(fig, animate_undamped, frames=600, interval=20, blit=True)
HTML(anim.to_html5_video())

C:\Users\xmc08\AppData\Local\Temp\ipykernel_6292\132928729.py:12: RuntimeWarning: overflow encountered in square
  T = (m_2 / l_t) * (np.square(l_f) * np.square(omega_h) + 2 * l_f * l_t * omega_h * omega_phi + np.square(l_t) * np.square(omega_phi)) # tension of the tibia swinging
C:\Users\xmc08\AppData\Local\Temp\ipykernel_6292\4022548555.py:2: RuntimeWarning: invalid value encountered in cos
  x = l_f * np.cos(theta_h)
C:\Users\xmc08\AppData\Local\Temp\ipykernel_6292\4022548555.py:3: RuntimeWarning: invalid value encountered in sin
  y = l_f * np.sin(theta_h) # y defined negative in order to hang leg down
C:\Users\xmc08\AppData\Local\Temp\ipykernel_6292\132928729.py:17: RuntimeWarning: invalid value encountered in scalar multiply
  new_omega_h = omega_h - (b_1 / I_1 * omega_h + (l_f * m_1 * g) / I_1 * np.cos(theta_h) - (l_f / I_1) * T) * t
C:\Users\xmc08\AppData\Local\Temp\ipykernel_6292\132928729.py:17: RuntimeWarning: invalid value encountered in cos
  new_omega_h = omega_h - (b_1 /